In [37]:
import requests
import pandas as pd
from dotenv import load_dotenv
import os
import json
import datetime
from io import StringIO
import boto3
load_dotenv()
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', None)

In [38]:
def extractData(symbol, interval, range):
    url = "https://yh-finance.p.rapidapi.com/stock/v3/get-chart"
    querystring = {  "region":"US", "lang":"en", "symbol": symbol, "interval":interval,
                   "range": range, "events":"capitalGain,div,split"}
    headers = {"X-RapidAPI-Key": os.environ.get("RAPIDAPI_KEY"),
    "X-RapidAPI-Host": os.environ.get("RAPIDAPI_HOST")}
    response = requests.request("GET", url, headers=headers, params=querystring)
    if response.ok:
        data = json.loads(response.content)
        return data["chart"]["result"][0]
    return None

response = extractData("V", "1m", "1d")
print(response)

{'meta': {'currency': 'USD', 'symbol': 'V', 'exchangeName': 'NYQ', 'instrumentType': 'EQUITY', 'firstTradeDate': 1205933400, 'regularMarketTime': 1677877202, 'gmtoffset': -18000, 'timezone': 'EST', 'exchangeTimezoneName': 'America/New_York', 'regularMarketPrice': 223.77, 'chartPreviousClose': 219.06, 'previousClose': 219.06, 'scale': 3, 'priceHint': 2, 'currentTradingPeriod': {'pre': {'timezone': 'EST', 'end': 1677853800, 'start': 1677834000, 'gmtoffset': -18000}, 'regular': {'timezone': 'EST', 'end': 1677877200, 'start': 1677853800, 'gmtoffset': -18000}, 'post': {'timezone': 'EST', 'end': 1677891600, 'start': 1677877200, 'gmtoffset': -18000}}, 'tradingPeriods': [[{'timezone': 'EST', 'end': 1677877200, 'start': 1677853800, 'gmtoffset': -18000}]], 'dataGranularity': '1m', 'range': '1d', 'validRanges': ['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y', 'ytd', 'max']}, 'timestamp': [1677853800, 1677853860, 1677853920, 1677853980, 1677854040, 1677854100, 1677854160, 1677854220, 167

In [39]:
def round_decimal(numbers):
    for i in range(len(numbers)):
        numbers[i] = round(numbers[i],2)
    return numbers

In [40]:
#Functions for creating dataframe from response object
def create_df_metadata(metadata_response):
    trading_periods = metadata_response["tradingPeriods"]
    start_time = datetime.datetime.fromtimestamp(trading_periods[0][0]['start']).strftime("%H:%M")
    end_time = datetime.datetime.fromtimestamp(trading_periods[0][0]['end']).strftime("%H:%M")
    period = start_time + " - " + end_time

    important_columns = {  "symbol": metadata_response["symbol"], 
                         "instrumentType":metadata_response["instrumentType"],
                         "regularMarketPrice":metadata_response["regularMarketPrice"],
                         "previousClose":metadata_response["previousClose"], "trade_period":period,
                         'timezone': metadata_response['timezone'], "range": metadata_response["range"],
                         "interval": metadata_response["dataGranularity"]}
    df_metadata = pd.DataFrame(important_columns, index =[0])

    return df_metadata

def create_df_timestamp(timestamp_response):
    timestamps = response["timestamp"]
    converted_timestamp = []
    for timestamp in timestamps:
        converted_timestamp.append(datetime.datetime.fromtimestamp(timestamp).strftime("%Y-%m-%d %H:%M:%S"))
    df_timestamp = pd.DataFrame({"timestamp": converted_timestamp})
    return df_timestamp


def create_df_indicators(indicators_response):
    indicators = response["indicators"]["quote"]
    indicators_dictionary = {"Volume": round_decimal(indicators[0]["volume"]),
                         "Open": round_decimal(indicators[0]["open"]),
                         "Close": round_decimal(indicators[0]["close"]),
                         "High": round_decimal(indicators[0]["high"]),
                         "Low": round_decimal(indicators[0]["low"])}

    df_indicators = pd.DataFrame(indicators_dictionary)

    return df_indicators
df_metadata = create_df_metadata(response["meta"])
print(df_metadata)

  symbol instrumentType  regularMarketPrice  previousClose   trade_period timezone range interval
0      V         EQUITY              223.77         219.06  09:30 - 16:00      EST    1d       1m


In [41]:
#Create dataframe for indicator and timestamp and merge all dataframe together
df_indicator = create_df_indicators(response["indicators"])
df_timestamp = create_df_timestamp(response["timestamp"])
df_merged = pd.concat([df_metadata["symbol"], df_indicator,df_timestamp], axis=1)
df_merged["symbol"] = merged_timestamp_indicators["symbol"].ffill()
print(df_merged)

    symbol  Volume    Open   Close    High     Low            timestamp
0        V  114079  220.20  220.36  220.54  219.90  2023-03-03 09:30:00
1        V   19257  220.33  220.34  220.45  220.10  2023-03-03 09:31:00
2        V   12650  220.37  220.18  220.37  220.11  2023-03-03 09:32:00
3        V   21735  220.20  220.24  220.38  220.12  2023-03-03 09:33:00
4        V   34754  220.24  220.80  220.84  220.23  2023-03-03 09:34:00
..     ...     ...     ...     ...     ...     ...                  ...
386      V   52870  223.94  223.84  224.01  223.82  2023-03-03 15:56:00
387      V   46600  223.84  223.88  223.94  223.83  2023-03-03 15:57:00
388      V   64575  223.88  223.80  223.89  223.62  2023-03-03 15:58:00
389      V   79526  223.80  223.71  223.88  223.68  2023-03-03 15:59:00
390      V       0  223.77  223.77  223.77  223.77  2023-03-03 16:00:00

[391 rows x 7 columns]


In [42]:
#Transform DF to CSV
def to_csv(df):
    df_buffer = StringIO()
    df.to_csv(df_buffer, index=False)
    df_csv = df_buffer.getvalue()
    return df_csv
metadata_csv = to_csv(df_metadata)
stockPrice_csv = to_csv(df_merged)


In [57]:
#Create S3 instance
s3_client = boto3.client("s3", region_name = 'us-east-1',
                            aws_access_key_id=os.environ.get("AWS_ACCESS_KEY"), 
                            aws_secret_access_key=os.environ.get("AWS_SECRET_ACCESSKEY"))
response = s3_client.list_buckets()
for bucket in response['Buckets']:
    print(f'  {bucket["Name"]}')

  ibmsalesproject
  stockds
